# Accretion disc

In this tutorial we set up a protoplanetary disc around a star represented by a sink particle, and we add a planet. This notebook generates a Phantom "temporary" dump file that can be read by Phantom as an initial condition. It also generates a Phantom "in" file. Together, these files can start a Phantom simulation.

## Initialization

First we import the required modules.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import phantomsetup

Here we set some constants for convenience.

In [ ]:
igas = phantomsetup.defaults.PARTICLE_TYPE['igas']

## Parameters

Now we set the parameters for the problem.

First is the `prefix` which sets the file name for the dump file and Phantom in file.

In [ ]:
prefix = 'disc'

### Resolution

We choose the resolution to be $10^6$ gas particles.

In [ ]:
number_of_particles = 1_000_000
particle_type = igas

### Viscosity

The SPH $\alpha$ viscosity parameter is its minimal value of 0.1.

In [ ]:
alpha_artificial = 0.1

### Units

We set the length and mass units to be au and solar masses, respectively. We will also set the time unit such that the gravitational constant is unity.

In [ ]:
length_unit = phantomsetup.units.unit_string_to_cgs('au')
mass_unit = phantomsetup.units.unit_string_to_cgs('solarm')
gravitational_constant = 1.0

### Star

The star is of solar mass, at the origin, with a 5 au accretion radius.

In [ ]:
stellar_mass = 1.0
stellar_accretion_radius = 5.0
stellar_position = (0.0, 0.0, 0.0)
stellar_velocity = (0.0, 0.0, 0.0)

### Disc

The disc has mass 0.01 solar masses, it extends from 10 au to 200 au.

In [ ]:
radius_min = 10.0
radius_max = 200.0

In [ ]:
disc_mass = 0.01

### Equation of state

The equation of state is locally isothermal. We set the aspect ratio H/R at a reference radius.

In [ ]:
ieos = 3
q_index = 0.75
aspect_ratio = 0.05
reference_radius = 10.0

### Planet

We add a planet at 100 au.

In [ ]:
planet_mass = 0.001
planet_position = (100.0, 0.0, 0.0)

In [ ]:
orbital_radius = np.linalg.norm(planet_position)
planet_velocity = np.sqrt(gravitational_constant * stellar_mass / orbital_radius)

We set the planet accretion radius as a fraction of the Hill sphere radius.

In [ ]:
planet_accretion_radius_fraction_hill_radius = 0.25

In [ ]:
planet_hill_radius = phantomsetup.orbits.hill_sphere_radius(
    orbital_radius, planet_mass, stellar_mass
)
planet_accretion_radius = (
    planet_accretion_radius_fraction_hill_radius * planet_hill_radius
)

### Surface density distribution

For the surface density distribution we use the Lynden-Bell and Pringle (1974) self-similar solution, i.e. a power law with an exponential taper.

In [ ]:
def density_distribution(radius, radius_critical, gamma):
    """Self-similar disc surface density distribution.

    This is the Lyden-Bell and Pringle (1974) solution, i.e. a power law
    with an exponential taper.
    """
    return phantomsetup.disc.self_similar_accretion_disc(radius, radius_critical, gamma)

radius_critical = 100.0
gamma = 1.0

In [ ]:
args = (radius_critical, gamma)

## Instantiate the `Setup` object

The following instantiates the `phantomsetup.Setup` object.

In [ ]:
setup = phantomsetup.Setup()

## Set attributes and add particles

### Prefix

Set the prefix.

In [ ]:
setup.prefix = prefix

### Units

Set units.

In [ ]:
setup.set_units(
    length=length_unit, mass=mass_unit, gravitational_constant_is_unity=True
)

### Equation of state

Set the equation of state. We get `polyk` from the aspect ratio parametrization.

In [ ]:
polyk = phantomsetup.eos.polyk_for_locally_isothermal_disc(
    q_index, reference_radius, aspect_ratio, stellar_mass, gravitational_constant
)

In [ ]:
setup.set_equation_of_state(ieos=ieos, polyk=polyk)

### Viscosity

Set the numerical viscosity to Phantom disc viscosity.

In [ ]:
setup.set_dissipation(disc_viscosity=True, alpha=alpha_artificial)

### Star

Add a star at the origin.

In [ ]:
setup.add_sink(
    mass=stellar_mass,
    accretion_radius=stellar_accretion_radius,
    position=stellar_position,
    velocity=stellar_velocity,
)

### Disc

Add the disc around the star.

In [ ]:
disc = phantomsetup.Disc()
disc.add_particles(
    particle_type=particle_type,
    number_of_particles=number_of_particles,
    disc_mass=disc_mass,
    density_distribution=density_distribution,
    radius_range=(radius_min, radius_max),
    q_index=q_index,
    aspect_ratio=aspect_ratio,
    reference_radius=reference_radius,
    stellar_mass=stellar_mass,
    gravitational_constant=gravitational_constant,
    args=(radius_critical, gamma),
)
setup.add_disc(disc)

### Planet

Add a planet in orbit around the star.

In [ ]:
setup.add_sink(
    mass=planet_mass,
    accretion_radius=planet_accretion_radius,
    position=planet_position,
    velocity=planet_velocity,
)

## Plot

Now we plot some quantities to see what we have set up.

First is the particles in the xy-plane. The sink particles are marked in red.

In [ ]:
fig, ax = plt.subplots()
ax.plot(setup.x[::10], setup.y[::10], 'k.', ms=0.5)
for sink in setup.sinks:
    ax.plot(sink.position[0], sink.position[1], 'ro')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_aspect('equal')

Next we plot the particles in the rz-plane.

In [ ]:
fig, ax = plt.subplots()
ax.plot(setup.R[::10], setup.z[::10], 'k.', ms=0.5)
ax.set_xlabel('$R$')
ax.set_ylabel('$z$')
ax.set_aspect('equal')
ax.set_ylim(bottom=2 * setup.z.min(), top=2 * setup.z.max())

Finally, we plot $v_{\phi}$ as a function of radius.

In [ ]:
fig, ax = plt.subplots()
ax.plot(setup.R[::10], setup.vphi[::10], 'k.', ms=0.5)
ax.set_xlabel('$R$')
ax.set_ylabel('$v_{\phi}$')

## Write to file

Now that we are happy with the setup, write the "temporary" dump file with the initial conditions and the Phantom "in" file.

First we set a working directory for the simulation.

In [ ]:
working_dir = '~/runs/disc'

In [ ]:
setup.write_dump_file(directory=working_dir)
setup.write_in_file(directory=working_dir)

## Compile Phantom

You can start a Phantom calculation from these two files but you must compile Phantom with the correct Makefile variables. We can use the `phantom_compile_command` method to show how Phantom would be compiled.

In [ ]:
print(setup.phantom_compile_command())

To compile Phantom we can use the `compile_phantom` method to compile Phantom. For example, the following will compile Phantom located in `phantom_dir` and copy the binary to `working_dir`.

```
result = setup.compile_phantom(
    phantom_dir='~/repos/phantom',
    working_dir=working_dir
)
```